In [1]:
# 필요한 라이브러리 Import
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
# 영화 예시 데이터 정의
examples = [
    {
        "title": "인셉션 (Inception)",
        "info": """감독: 크리스토퍼 놀란
주연: 레오나르도 디카프리오, 조셉 고든 레빗, 엘렌 페이지
예산: $160,000,000
수익: $836,836,967
장르: SF, 액션, 스릴러
시놉시스: 꿈의 세계에서 타인의 생각을 훔치는 특수 보안 전문가가 '인셉션'이라는 
생각을 심는 미션을 수행하게 되면서 벌어지는 이야기"""
    },
    {
        "title": "어벤져스: 엔드게임 (Avengers: Endgame)",
        "info": """감독: 안소니 루소, 조 루소
주연: 로버트 다우니 주니어, 크리스 에반스, 스칼렛 요한슨
예산: $356,000,000
수익: $2,797,501,328
장르: 액션, SF, 어드벤처
시놉시스: 타노스의 핑거스냅으로 절반의 생명이 사라진 후, 남은 어벤져스 멤버들이 
동료들을 되살리기 위한 마지막 역전의 기회에 도전하는 이야기"""
    },
    {
        "title": "기생충 (Parasite)",
        "info": """감독: 봉준호
주연: 송강호, 조여정, 최우식, 박소담
예산: $11,400,000
수익: $258,773,645
장르: 드라마, 스릴러, 블랙 코미디
시놉시스: 전원 백수인 기택네 가족이 부유한 박사장네 가족의 집에 하나둘 침투하며 
벌어지는 예측할 수 없는 이야기"""
    }
]

In [3]:
# 예시 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "영화 {title}에 대해 알려주세요."),
    ("assistant", "{info}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 영화 정보를 제공하는 전문가입니다. 
    요청받은 영화에 대해 다음 형식으로 정보를 제공해주세요:
    
    감독: [감독 이름]
    주연: [주요 배우들]
    예산: [제작 예산]
    수익: [전세계 흥행 수익]
    장르: [영화 장르]
    시놉시스: [간단한 줄거리]
    """),
    few_shot_prompt,
    ("human", "영화 {input}에 대해 알려주세요."),
])

In [ ]:
# 스트리밍 설정으로 모델 초기화
model = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 체인 구성
chain = final_prompt | model

In [5]:
# 영화 정보 요청
response = chain.invoke({"input": "매트릭스 (The Matrix)"})

감독: 라나 워쇼스키, 리리 워쇼스키
주연: 키아누 리브스, 로리 앤서니, 캐리 앤 모스
예산: $63,000,000
수익: $463,517,383
장르: 액션, SF
시놉시스: 현실과 가상 세계의 경계가 모호해진 미래에서, 해커 네오가 인류를 지배하는 
기계들과의 전쟁에 휘말리며 진정한 현실을 깨닫고 인류의 구원자로 성장하는 이야기

In [6]:
# 영화 정보 요청
response = chain.invoke({"input": "해리포터 (Harry Potter)"})

해리 포터 시리즈는 여러 편으로 구성되어 있으므로, 첫 번째 영화인 "해리 포터와 마법사의 돌"에 대한 정보를 제공하겠습니다.

감독: 크리스 콜럼버스  
주연: 다니엘 래드클리프, 엠마 왓슨, 루퍼트 그린트  
예산: $125,000,000  
수익: $974,755,371  
장르: 판타지, 어드벤처  
시놉시스: 평범한 소년 해리 포터가 자신의 마법사 혈통을 알게 되고, 호그와트 마법학교에 입학하여 친구들과 함께 마법의 세계에서 모험을 겪으며 악의 세력과 맞서 싸우는 이야기입니다.